<a href="https://colab.research.google.com/github/AfshinRezakhani/Thesis1/blob/main/GAN_GRU_d8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import GRU, Dense, Input, Reshape, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd

df = pd.read_csv("/content/d8.csv")
X = df.drop(columns=["F"]).values
y = df["F"].values

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

latent_dim = 20

def build_generator():
    inputs = Input(shape=(latent_dim,))
    x = Dense(32, activation='relu')(inputs)
    x = Reshape((1, 32))(x)
    x = GRU(64, return_sequences=True)(x)
    x = GRU(64)(x)
    x = Dropout(0.3)(x)
    outputs = Dense(X_train.shape[1])(x)
    return Model(inputs, outputs, name="Generator")

def build_discriminator():
    inputs = Input(shape=(X_train.shape[1],))
    x = Dense(32, activation='relu')(inputs)
    x = Reshape((1, 32))(x)
    x = GRU(64, return_sequences=True)(x)
    x = GRU(64)(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    return Model(inputs, outputs, name="Discriminator")

generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

discriminator.trainable = False
z = Input(shape=(latent_dim,))
generated_data = generator(z)
disc_output = discriminator(generated_data)
gan = Model(z, disc_output)
gan.compile(optimizer='adam', loss='binary_crossentropy')

def train_gan(epochs=1500, batch_size=64):
    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_samples = X_train[idx]

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_samples = generator.predict(noise)

        X_combined = np.vstack((real_samples, fake_samples))
        y_combined = np.hstack((np.ones(batch_size), np.zeros(batch_size)))

        d_loss = discriminator.train_on_batch(X_combined, y_combined)

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = gan.train_on_batch(noise, np.ones(batch_size))

        if epoch % 100 == 0:
            print(f"Epoch {epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}")

train_gan()

num_samples_to_generate = abs(np.sum(y_train == 0) - np.sum(y_train == 1))
noise = np.random.normal(0, 1, (num_samples_to_generate, latent_dim))
synthetic_data = generator.predict(noise)
synthetic_labels = np.ones(num_samples_to_generate)  # Generate only class 1 samples

X_train_balanced = np.vstack((X_train, synthetic_data))
y_train_balanced = np.hstack((y_train, synthetic_labels))

balanced_df = pd.DataFrame(X_train_balanced, columns=df.columns[:-1])
balanced_df["F"] = y_train_balanced
balanced_df.to_csv("/content/d8_balancedwithGRU.csv", index=False)

classifier = tf.keras.Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier.fit(X_train_balanced, y_train_balanced, epochs=100, batch_size=64, verbose=1)

y_pred = classifier.predict(X_test).round()
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Accuracy: {accuracy}, F1-score: {f1}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step 


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 0, D Loss: 0.6936993598937988, G Loss: 0.6928857564926147
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
2/2 ━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


746/746 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6562 - loss: 0.5912
Epoch 2/100
746/746 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7013 - loss: 0.5417
Epoch 3/100
746/746 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7074 - loss: 0.5396
Epoch 4/100
746/746 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7041 - loss: 0.5426
Epoch 5/100
746/746 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7070 - loss: 0.5380
Epoch 6/100
746/746 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7065 - loss: 0.5399
Epoch 7/100
746/746 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7038 - loss: 0.5407
Epoch 8/100
746/746 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7082 - loss: 0.5387
Epoch 9/100
746/746 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7056 - loss: 0.5407
Epoch 10/100
746/746 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7062 - loss: 0.5391
Epoch 11/100
746/746 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7023 - loss: 0.5421
Epoch 12/100
746/746 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step